In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.common.exceptions import TimeoutException, WebDriverException,ElementNotVisibleException
from webdriver_manager.chrome import ChromeDriverManager
from random_user_agent.user_agent import UserAgent 
from random_user_agent.params import SoftwareName, OperatingSystem 
from time import sleep
import logging
import traceback
from self import self

class Request:
    
    selenium_retries=0
    nr=1
    logging.basicConfig(filename="TheScraping.log",
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    filemode='w')
    
    logger=logging.getLogger('wine_project.requests')
    logger.setLevel(logging.INFO) 
  
    def __init__(self,url):
        self.url = url
   
    def get_selenium_res(self,class_name):
        self.logger.info('sending request #: '+str(self.nr)+' to '+str(self.url))
        self.nr+=1
        try:
            software_names=[SoftwareName.CHROME.value]
            operating_systems=[OperatingSystem.WINDOWS.value,
                                  OperatingSystem.LINUX.value]
            user_agent_rotator=UserAgent(software_names=software_names,
                                            operating_systems=operating_systems,
                                            limit=100)
            user_agent=user_agent_rotator.get_random_user_agent()
                
            chrome_options=Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--window-size=1420,1080")
            chrome_options.add_argument(f'user-agent={user_agent}')
                
            browser = webdriver.Chrome('/Users/lilly/Desktop/chromedriver',options=chrome_options,service_args=["--verbose", "--log-path=/Users/lilly/Documents/ISDS2020/Exercises/Vin.log"])
            
            browser.get(self.url)
                
            time_to_wait=90
            
            WebDriverWait(browser,time_to_wait).until(
            EC.presence_of_element_located((By.CLASS_NAME, class_name)))
                
        except(TimeoutException,WebDriverException):
            self.logger.error(traceback.format_exc())
            sleep(6)
            self.selenium_retries += 1
            self.logger.info('Selenium retry #: '+ str(self.selenium_retries))
            return self.get_selenium_res(class_name)
        else:
            browser.maximize_window()
            page_html = browser.page_source
            browser.close()
            self.logger.info('Success')
            return page_html
        

In [4]:
import pandas as pd

#Henter csv fil med vin data
vivino=pd.read_csv('vivino_v7.csv',index_col=[0])

#Drop duplicates by link
vivino=vivino.drop_duplicates(['link']).reset_index(drop=True)

len(vivino)

8385

In [17]:
from bs4 import BeautifulSoup
import re
import numpy as np


B=[]

for a in vivino['link'][2030:2050]:
    A=[]

    soup = BeautifulSoup(Request(a).get_selenium_res('indicatorBar__progress--3aXLX'), 'lxml')
    
    #Indhenter gennemsnitsprisen
    p=soup.find_all('span', {'class': 'purchaseAvailabilityPPC__amount--2_4GT'})

    if not p:
        g=soup.find_all('span', {'class': 'purchaseAvailability__currentPrice--3mO4u'})
        if not g:
            A.append(np.nan)
        else:
            A.append(soup.find_all('span', {'class': 'purchaseAvailability__currentPrice--3mO4u'})[0].text)
    else:
        A.append(soup.find_all('span', {'class': 'purchaseAvailabilityPPC__amount--2_4GT'})[0].text)
    #Indhenter antal ratings
    ratings=soup.find_all('div', {'class': 'vivinoRatingWide__basedOn--s6y0t'})[0].text

    if not ratings:
        A.append(np.nan)
    else: 
        A.append(re.sub(r'[^\d]','',ratings))
    # Indhenter antal brugerbedømmgelser 
    reviews=soup.find_all('div', {'class': 'tasteCharacteristics__averageOfReviews--kut7x'})[0].text

    if not reviews:
        A.append(np.nan)
    else: 
        n=re.search(r'(på)',reviews).span(0)[1]
        m=reviews[n:]
        A.append(re.search(r'^(.+?)[a-zA-Z]', m)[1])
        
    
    # Indhenter smagsnoter
    
    bar = soup.find_all('span', {'class': 'indicatorBar__progress--3aXLX'})
                 
    if not bar:
        for i in range(0,4):
            A.append(np.nan)
    else:
        for i in range(0,4):
            A.append(re.sub(r'[^\d\.]','',bar[i]['style'][18:]))
    

    #Indhenter alcohol%
    script = str(soup.find_all("script"))
    n=[]
    for match in re.finditer(r'("alcohol":)',script):
        n.append(match.span(0))
    span=len(n)

    if span==0 or span==1:
        A.append(np.nan)
    else:
        m=script[n[span-1][span-1]:]
        if bool(re.match(r'(minimum)', re.search(r'^(.+?),', m)[1]))==True:
            A.append(np.nan)
        else:
            A.append(re.sub(r'\}','',re.search(r'^(.+?),', m)[1]))

    #Indhenter sukker pr. gram
    if re.search(r'(residual_sugar_grams_per_liter)',script)==None:
        A.append(np.nan)
    else:
        n=re.search(r'(residual_sugar_grams_per_liter)',script).span(0)[1]
        m=script[ n+2:]
        A.append(re.search(r'^(.+?),', m)[1])

    # Indhenter syre pr.liter
    if re.search(r'(acidity_grams_per_liter)',script)==None:
        A.append(np.nan)
    else:
        n=re.search(r'(acidity_grams_per_liter)',script).span(0)[1]
        m=script[n+2:]
        A.append(re.search(r'^(.+?),', m)[1])

    # Winery
    A.append(re.sub("(\n)", "", soup.find_all('a', {'class': 'winery'})[0].text))
    
    B.append(A)

KeyboardInterrupt: 

In [18]:
df2

,price,total_ratings,total_reviews,light-bold,smooth-tannic,dry-sweet,soft-acidic,alcohol_pct,residual_sugar_g/l,acidity_g/l,winery
0,2050.86 kr.,75,11.0,72.2130,40.0278,18.00000,20.8148,NaN,NaN,NaN,Marion
1,11500 kr.,49,405.0,61.6076,80.2688,9.65777,82.9995,NaN,NaN,NaN,Giacomo Conterno
2,1100 kr.,40,83.0,85.0000,27.6326,46.49700,27.5195,NaN,NaN,NaN,Tedeschi
3,1251.36 kr.,37,382.0,61.3094,77.8291,7.14819,82.4921,NaN,NaN,NaN,Vietti
4,825 kr.,30,24.0,85.0000,19.9018,47.92760,20.3641,NaN,NaN,NaN,Ernesto Ruffo
...,...,...,...,...,...,...,...,...,...,...,...
2020,245.06 kr.,50,47.0,84.2333,26.4913,47.45000,25.4688,NaN,NaN,NaN,Le Calendre
2021,445.42 kr.,49,121.0,67.1103,80.0113,12.03880,85.0000,NaN,NaN,NaN,Manzone
2022,209.17 kr.,50,12.0,68.3750,39.0625,30.00000,21.6875,NaN,NaN,NaN,Tenuta il Sogno
2023,247 kr.,49,138.0,85.0000,23.2803,46.84700,22.3468,NaN,NaN,NaN,Aldegheri


In [ ]:
print(len(B))
# Creating dataframe to merge and save:
df=pd.DataFrame(B, columns=['price', 'total_ratings','total_reviews','light-bold','smooth-tannic','dry-sweet','soft-acidic','alcohol_pct','residual_sugar_g/l','acidity_g/l','winery'])
merged=pd.concat([df2,df]).reset_index(drop=True)
df2=merged
df2.to_csv('8384.csv')

#Husk at droppe link 1866,2025,2026 i vivino dataframe inden merge, da denne er udeladet
df2